In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import cv2
import numpy as np
from PIL import Image, ImageDraw
lmap={
    'potatoe':'Potato',
    'maize':'Maize',
    'paddy':'Paddy'
}


# Set the path to your JSON file and the output folder
file = "/content/drive/MyDrive/Jsonn_and_csv/project-24-at-2024-12-21-15-01-93d36c5d.json"
output_folder = os.path.join('/content/drive/MyDrive/', "jsontopng")
print("Current working directory:",'/content/drive/MyDrive/')


os.makedirs(output_folder, exist_ok=True)
data = json.load(open(file, 'r'))

def create_mask(width, height, polygons):
  mask = Image.new('L', (width, height), 0)
  draw = ImageDraw.Draw(mask)
  for polygon in polygons:
    if len(polygon) == 1:  # If only one point in the polygon
      x, y = polygon[0]
      polygon_tuples = [(int(x * width/100), int(y * height/100)), (int(x * width/100) + 1, int(y * height/100) + 1)]  # Create a tiny rectangle
      draw.polygon(polygon_tuples, fill=255)
    else:
      # Convert polygon coordinates to a list of (x, y) tuples with integer coordinates
      polygon_tuples = [(int(x * width/100), int(y * height/100)) for x, y in polygon]
      draw.polygon(polygon_tuples, fill=255)  # Pass the converted coordinates to draw.polygon
  print(polygon_tuples)
  return np.array(mask)

for element in data:
  annotations = element.get('annotations', [])
  if annotations:
    print(f"got {len(annotations)} annotations")
    original_width = None
    original_height = None

    label_masks = {}

    for annotation in annotations:
      results = annotation.get('result', [])
      print(f"\tGot {len(results)} results")
      for result in results:
        if original_width is None:
          original_width = result.get('original_width')
          original_height = result.get('original_height')
          print(f"\t\tOriginal width: {original_width}, Original height: {original_height}")

        if 'value' in result and 'polygonlabels' in result['value']:
          value = result['value']['points']
          labels = result['value']['polygonlabels']
          print(f"\t\tValue: {value}, Labels: {labels}")
          for label in labels:
            if label not in label_masks:
              label_masks[label] = []
            label_masks[label].append(value)

    for label, polygons in label_masks.items():
      mask = create_mask(original_width, original_height, polygons)
      filename = f" task-{element['id']}-annotation-{annotation['id']}-by-1-tag-{lmap[label]}-0.png"  # Modified filename
      cv2.imwrite(os.path.join(output_folder, filename), mask)

Current working directory: /content/drive/MyDrive/
got 1 annotations
	Got 8 results
		Original width: 512, Original height: 512
		Value: [[1.773937724350947, 99.58260288838802], [7.304449453209781, 99.79130144419402], [8.974037899657732, 94.57383754904416], [10.226229234493694, 81.73887636697555], [10.747975624008678, 70.99090074296687], [10.330578512396691, 61.91251356540614], [10.539277068202685, 52.834126387845416], [4.695717505634859, 40.93830870690377], [2.504382669671925, 40.41656231738878], [1.4608898906419563, 27.894648969029156], [0.31304783370899064, 26.40036730945821], [0.20869855580599375, 46.33107938893062], [0.20869855580599375, 54.47032306536437], [2.9217797812839126, 61.77477251857415], [1.878287002253944, 65.42699724517905], [1.878287002253944, 69.91401619500792], [0.5217463895149844, 74.71408297854578], [0.939143501126972, 79.75624008681861], [0.939143501126972, 90.6085649887303]], Labels: ['paddy']
		Value: [[11.895817680941645, 99.06085649887304], [11.68711912513565